In [61]:
# Sushma Chowdary Aari
# 11713230

import random
import heapq
import matplotlib.pyplot as plt

# Define a base class for search problems
class SearchProblem(object):
    def starting_node(self):
        raise NotImplementedError("starting_node")

    def goal_is(self, node):
        raise NotImplementedError("goal_is")

    def neighbors(self, node):
        raise NotImplementedError("neighbors")

    def heuristic_method(self, n):
        return 0

# Define an arc class to represent connections between nodes in a search problem
class Arc(object):
    def __init__(self_in, from_node, to_node, cost=1, action=None):
        assert cost >= 0, ("Cost cannot be negative for" + str(from_node) + "->" + str(to_node) + ", cost: " + str(cost))
        self_in.from_node = from_node
        self_in.to_node = to_node
        self_in.action = action
        self_in.cost = cost

    def __repr__(self_in):
        if self_in.action:
            return str(self_in.from_node) + " --" + str(self_in.action) + "--> " + str(self_in.to_node)
        else:
            return str(self_in.from_node) + " --> " + str(self_in.to_node)

# Define a base class for objects that can be displayed
class Displayable(object):
    max_display_level = 1

    def display(self, level, *args, **kwargs):
        if level <= self.max_display_level:
            print(*args, **kwargs)

# Define a class for simple local search
class SimpleLocalSearch(Displayable):
    def __init__(self_in, csp):
        self_in.csp = csp
        self_in.variables_to_select = {var for var in self_in.csp.variables if len(var.domain) > 1}
        self_in.current_assignment = None
        self_in.number_of_steps = 0

    def restart(self_in):
        self_in.current_assignment = {var: random_choice(var.domain) for var in self_in.csp.variables}
        ca = self_in.current_assignment
        self_in.conflicts = set()
        for con in self_in.csp.constraints:
            if not con.holds(self_in.current_assignment):
                self_in.conflicts.add(con)
                ca2 = len(self_in.conflicts)
                self_in.variable_priority_queue = None

    def search(self, max_steps, prob_best=0, prob_anycon=1.0):
        if self.current_assignment is None:
            self.restart()
        self.number_of_steps += 1
        if not self.conflicts:
            ca3 = self.current_assignment
            print(1, "Solution found:", ca3, "after restart")
            return self.number_of_steps

        if prob_best > 0:
            return self.search_with_var_pq(max_steps, prob_best, prob_anycon)
        else:
            return self.search_with_any_conflict(max_steps, prob_anycon)

    def search_with_var_pq(self, max_steps, prob_best=1.0, prob_anycon=1.0):
        if not self.variable_priority_queue:
            self.create_priority_queue()
        pick_best_or_con = prob_best + prob_anycon
        for i in range(max_steps):
            self.number_of_steps += 1
            randnum = random.random()
            if randnum < prob_best:
                var, oldval = self.variable_priority_queue.top()
            elif randnum < pick_best_or_con:
                con = random_choice(self.conflicts)
                var = random_choice(con.scope)
            else:
                var = random_choice(self.variables_to_select)
            if len(var.domain) > 1:
                val = random_choice([val for val in var.domain if val is not self.current_assignment[var]])
                print("Assigning", var, val)
                var_differential = {}
                self.current_assignment[var] = val
                for varcon in self.csp.var_to_const[var]:
                    if varcon.holds(self.current_assignment):
                        if varcon in self.conflicts:
                            self.conflicts.remove(varcon)
                            for v in varcon.scope:
                                var_differential[v] = var_differential.get(v, 0) - 1
                        else:
                            if varcon not in self.conflicts:
                                self.conflicts.add(varcon)
                                for v in varcon.scope:
                                    var_differential[v] = var_differential.get(v, 0) + 1
                self.variable_priority_queue.update_each_priority(var_differential)
                ca4 = len(self.conflicts)
                print("Number of conflicts", ca4)
                if not self.conflicts:
                    self.display(1, "Solution found:", self.current_assignment, "in", self.number_of_steps, "steps")
                    return self.number_of_steps
        self.display(1, "No solution in", self.number_of_steps, "steps", len(self.conflicts), "conflicts remain")
        return None

    def create_priority_queue(self):
        self.variable_priority_queue = UpdatablePriorityQueue()
        var_to_number_conflicts = {}
        for con in self.conflicts:
            for var in con.scope:
                var_to_number_conflicts[var] = var_to_number_conflicts.get(var, 0) + 1
        for var, num in var_to_number_conflicts.items():
            if num > 0:
                self.variable_priority_queue.add(var, -num)

# Define a function for making a random choice from a set
def random_choice(st):
    return random.choice(list(st))

# Define a class for an updatable priority queue
class UpdatablePriorityQueue(object):
    def __init__(self):
        self.pq = []
        self.elt_map = {}
        self.REMOVED = "*removed*"
        self.max_size = 0

    def add(self, elt, val):
        assert val <= 0, val
        assert elt not in self.elt_map, elt
        new_triple = [val, random.random(), elt]
        heapq.heappush(self.pq, new_triple)
        self.elt_map[elt] = new_triple

    def remove(self, elt):
        if elt in self.elt_map:
            self.elt_map[elt][2] = self.REMOVED
            del self.elt_map[elt]

    def update_each_priority(self, update_dict):
        for elt, incr in update_dict.items():
            if incr != 0:
                newval = self.elt_map.get(elt, [0])[0] - incr
                assert newval <= 0, str(elt) + ":" + str(newval + incr) + "-" + str(incr)
                self.remove(elt)
                if newval != 0:
                    self.add(elt, newval)

    def pop(self):
        self.max_size = max(self.max_size, len(self.pq))
        triple = heapq.heappop(self.pq)
        while triple[2] == self.REMOVED:
            triple = heapq.heappop(self.pq)
        del self.elt_map[triple[2]]
        return triple[2], triple[0]

    def top(self):
        self.max_size = max(self.max_size, len(self.pq))
        triple = self.pq[0]
        while triple[2] == self.REMOVED:
            heapq.heappop(self.pq)
            triple = self.pq[0]
        return triple[2], triple[0]

    def empty(self):
        return all(triple[2] == self.REMOVED for triple in self.pq)

# Define a class for a variable in a constraint satisfaction problem
class Variable(object):
    def __init__(self, name, domain, position=None):
        self.name = name
        self.domain = domain
        self.position = position if position else (random.random(), random.random())
        self.size = len(domain)

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

# Define a class for a constraint in a constraint satisfaction problem
class Constraint(object):
    def __init__(self, scope, condition, string=None, position=None):
        self.scope = scope
        self.condition = condition
        if string is None:
            self.string = self.condition.__name__ + str(self.scope)
        else:
            self.string = string
        self.position = position

    def can_evaluate(self, assignment):
        return all(v in assignment for v in self.scope)

    def holds(self, assignment):
        return self.condition(*tuple(assignment[v] for v in self.scope))

# Define a class for a constraint satisfaction problem
class CSP(object):
    def __init__(self, title, variables, constraints):
        self.title = title
        self.variables = variables
        self.constraints = constraints
        self.var_to_const = {var: set() for var in self.variables}
        for con in constraints:
            for var in con.scope:
                self.var_to_const[var].add(con)

    def consistent(self, assignment):
        return all(con.holds(assignment) for con in self.constraints if con.can_evaluate(assignment))

    def show(self):
        plt.ion()
        ax = plt.figure().gca()
        ax.set_axis_off()
        plt.title(self.title)
        var_bbox = dict(boxstyle="round4,pad=1.0,rounding_size=0.5")
        con_bbox = dict(boxstyle="square,pad=1.0", color="green")
        for var in self.variables:
            if var.position is None:
                var.position = (random.random(), random.random())
        for con in self.constraints:
            if con.position is None:
                con.position = tuple(sum(var.position[i] for var in con.scope) / len(con.scope) for i in range(2))
            bbox = dict(boxstyle="square,pad=1.0", color="green")
            for var in con.scope:
                ax.annotate(con.string, var.position, xytext=con.position, arrowprops={'arrowstyle': '-'}, bbox=con_bbox,
                            ha='center')
        for var in self.variables:
            x, y = var.position
            plt.text(x, y, var.name, bbox=var_bbox, ha='center')

# Define a function to create a constraint for meeting at a certain position
def meet_at(p1, p2):
    def meets(w1, w2):
        return w1[p1] == w2[p2]

    meets.__name__ = "meet_at(" + str(p1) + ',' + str(p2) + ')'
    return meets

# Define some variables and constraints for a crossword puzzle
one_down = Variable('one_down', {'dance', 'usage', 'first', 'loses', 'fuels', 'haste', 'given', 'sense', 'sound', 'think'}, position=(0.3, 0.9))
two_across =Variable('two_across', {'eta', 'hat', 'her', 'him', ' one'}, position=(0.1, 0.7))
three_across = Variable('three_across', {'at', 'be', 'he', 'it', 'on'}, position=(0.1, 0.3))
four_across = Variable('four_across', {'dance', 'usage', 'first', 'loses', 'fuels', 'haste', 'given', 'sense', 'sound', 'think'}, position=(0.4, 0.6))
five_down = Variable('five_down', {'dance', 'usage', 'first', 'loses', 'fuels', 'haste', 'given', 'sense', 'sound', 'think'}, position=(0.3, 0.0))
six_across = Variable('six_across', {'desk', 'easy', 'dove', 'else', 'help', 'kind', 'soon', 'this'}, position=(0.7, 0.8))

crossword1 = CSP("crossword1",
                {one_down, two_across, three_across, four_across, five_down, six_across},
                [
                    Constraint([three_across, one_down], meet_at(0, 1)),
                    Constraint([four_across, one_down], meet_at(1, 2)),
                    Constraint([one_down, six_across], meet_at(4, 0)),
                    Constraint([four_across, five_down], meet_at(3, 0)),
                    Constraint([six_across, five_down], meet_at(2, 2)),
                ])

searcher = SimpleLocalSearch(crossword1)
print(searcher.search(300, 1.0))


Assigning one_down think
Number of conflicts 5
Assigning one_down given
Number of conflicts 4
Assigning five_down first
Number of conflicts 4
Assigning five_down given
Number of conflicts 3
Assigning four_across first
Number of conflicts 3
Assigning four_across dance
Number of conflicts 3
Assigning four_across first
Number of conflicts 3
Assigning four_across usage
Number of conflicts 2
Assigning one_down first
Number of conflicts 3
Assigning one_down think
Number of conflicts 3
Assigning one_down sound
Number of conflicts 2
Assigning one_down fuels
Number of conflicts 2
Assigning one_down haste
Number of conflicts 1
Assigning three_across he
Number of conflicts 1
Assigning three_across on
Number of conflicts 1
Assigning three_across he
Number of conflicts 1
Assigning three_across it
Number of conflicts 1
Assigning three_across at
Number of conflicts 0
Solution found: {three_across: 'at', one_down: 'haste', six_across: 'dove', four_across: 'usage', two_across: ' one', five_down: 'given